In [2]:
import pickle
from dataclasses import fields
from pathlib import Path

import igl
import numpy as np
import pyvista as pv

from cardiac_electrophysiology import mesh_processing as mp
from cardiac_electrophysiology.ulac import base, ulac

In [3]:
patient_id = "01"
input_mesh_file = Path(f"../data/processed/patient_{patient_id}/mesh_with_fibers_tags.vtk")
segmentation_file_name = Path(f"../data/processed/patient_{patient_id}/segmentation.pkl")
anatomical_tags = ulac.FeatureTags(MV=0, LAA=1, LIPV=2, LSPV=3, RIPV=4, RSPV=5)

mesh = pv.read(input_mesh_file)
mesh = mp.convert_unstructured_to_polydata_mesh(mesh)

In [4]:
# segmentation_paths = ulac.construct_paths(mesh, anatomical_tags)

# segmentation_file_name.parent.mkdir(parents=True, exist_ok=True)
# with segmentation_file_name.open("wb") as f:
#     pickle.dump(segmentation_paths, f)

In [5]:
with segmentation_file_name.open("rb") as f:
    segmentation_paths = pickle.load(f)

In [6]:
markers = ulac.get_marker_points(segmentation_paths)
parameterized_paths = ulac.parameterize_paths(mesh, segmentation_paths, markers)
uac_paths = ulac.construct_uac_boundaries(parameterized_paths, markers)
path_boundaries = ulac.get_patch_boundaries(uac_paths)

In [7]:
# Roof
roof_boundary = path_boundaries.roof.inds
splitted_meshes = base.split_from_enclosing_boundary(mesh, roof_boundary)
roof_mesh = splitted_meshes[1]

vertices = np.array(mesh.points[roof_mesh.point_inds])
simplices = roof_mesh.connectivity
boundary_inds = np.array([np.where(roof_mesh.point_inds == ind)[0][0] for ind in roof_boundary])
uac_coordinates = np.hstack(
    (path_boundaries.roof.alpha[:, None], path_boundaries.roof.beta[:, None])
)

harmonic_map = igl.harmonic(
    V=vertices,
    F=simplices,
    b=boundary_inds,
    bc=uac_coordinates,
    k=1,
)
harmonic_map_z = np.zeros((harmonic_map.shape[0], 1))
harmonic_map_3d = np.hstack((harmonic_map, harmonic_map_z))
roof_uac_mesh = pv.PolyData.from_regular_faces(harmonic_map_3d, simplices)


# Anterior
anterior_boundary = path_boundaries.anterior.inds
splitted_meshes = base.split_from_enclosing_boundary(mesh, anterior_boundary)
anterior_mesh = splitted_meshes[0]

vertices = np.array(mesh.points[anterior_mesh.point_inds])
simplices = anterior_mesh.connectivity
boundary_inds = np.array(
    [np.where(anterior_mesh.point_inds == ind)[0][0] for ind in anterior_boundary]
)
uac_coordinates = np.hstack(
    (path_boundaries.anterior.alpha[:, None], path_boundaries.anterior.beta[:, None])
)

harmonic_map = igl.harmonic(
    V=vertices,
    F=simplices,
    b=boundary_inds,
    bc=uac_coordinates,
    k=1,
)
harmonic_map_z = np.zeros((harmonic_map.shape[0], 1))
harmonic_map_3d = np.hstack((harmonic_map, harmonic_map_z))
anterior_uac_mesh = pv.PolyData.from_regular_faces(harmonic_map_3d, simplices)


# Septal
septal_boundary = path_boundaries.septal.inds
splitted_meshes = base.split_from_enclosing_boundary(mesh, septal_boundary)
septal_mesh = splitted_meshes[1]

vertices = np.array(mesh.points[septal_mesh.point_inds])
simplices = septal_mesh.connectivity
boundary_inds = np.array([np.where(septal_mesh.point_inds == ind)[0][0] for ind in septal_boundary])
uac_coordinates = np.hstack(
    (path_boundaries.septal.alpha[:, None], path_boundaries.septal.beta[:, None])
)

harmonic_map = igl.harmonic(
    V=vertices,
    F=simplices,
    b=boundary_inds,
    bc=uac_coordinates,
    k=1,
)
harmonic_map_z = np.zeros((harmonic_map.shape[0], 1))
harmonic_map_3d = np.hstack((harmonic_map, harmonic_map_z))
septal_uac_mesh = pv.PolyData.from_regular_faces(harmonic_map_3d, simplices)

In [8]:
# Lateral
lateral_boundary = path_boundaries.lateral.inds
splitted_meshes = base.split_from_enclosing_boundary(mesh, lateral_boundary)
lateral_mesh = splitted_meshes[1]

vertices = np.array(mesh.points[lateral_mesh.point_inds])
simplices = lateral_mesh.connectivity
boundary_inds = np.array([np.where(lateral_mesh.point_inds == ind)[0][0] for ind in lateral_boundary])
uac_coordinates = np.hstack(
    (path_boundaries.lateral.alpha[:, None], path_boundaries.lateral.beta[:, None])
)

harmonic_map = igl.harmonic(
    V=vertices,
    F=simplices,
    b=boundary_inds,
    bc=uac_coordinates,
    k=1,
)
harmonic_map_z = np.zeros((harmonic_map.shape[0], 1))
harmonic_map_3d = np.hstack((harmonic_map, harmonic_map_z))
lateral_uac_mesh = pv.PolyData.from_regular_faces(harmonic_map_3d, simplices)

In [9]:
# Posterior
posterior_boundary = path_boundaries.posterior.inds
splitted_meshes = base.split_from_enclosing_boundary(mesh, posterior_boundary)
posterior_mesh = splitted_meshes[1]

vertices = np.array(mesh.points[posterior_mesh.point_inds])
simplices = posterior_mesh.connectivity
boundary_inds = np.array([np.where(posterior_mesh.point_inds == ind)[0][0] for ind in posterior_boundary])
uac_coordinates = np.hstack(
    (path_boundaries.posterior.alpha[:, None], path_boundaries.posterior.beta[:, None])
)

harmonic_map = igl.harmonic(
    V=vertices,
    F=simplices,
    b=boundary_inds,
    bc=uac_coordinates,
    k=1,
)
harmonic_map_z = np.zeros((harmonic_map.shape[0], 1))
harmonic_map_3d = np.hstack((harmonic_map, harmonic_map_z))
posterior_uac_mesh = pv.PolyData.from_regular_faces(harmonic_map_3d, simplices)

In [14]:
uac_rspv_inner_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.RSPV_inner.alpha[:, None],
        uac_paths.pv_boundaries.RSPV_inner.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.RSPV_inner.alpha.shape[0], 1)),
    )
)
uac_rspv_outer_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.RSPV_outer.alpha[:, None],
        uac_paths.pv_boundaries.RSPV_outer.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.RSPV_outer.alpha.shape[0], 1)),
    )
)
uac_lspv_inner_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.LSPV_inner.alpha[:, None],
        uac_paths.pv_boundaries.LSPV_inner.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.LSPV_inner.alpha.shape[0], 1)),
    )
)
uac_lspv_outer_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.LSPV_outer.alpha[:, None],
        uac_paths.pv_boundaries.LSPV_outer.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.LSPV_outer.alpha.shape[0], 1)),
    )
)
uac_lipv_inner_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.LIPV_inner.alpha[:, None],
        uac_paths.pv_boundaries.LIPV_inner.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.LIPV_inner.alpha.shape[0], 1)),
    )
)
uac_lipv_outer_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.LIPV_outer.alpha[:, None],
        uac_paths.pv_boundaries.LIPV_outer.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.LIPV_outer.alpha.shape[0], 1)),
    )
)
uac_ripv_inner_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.RIPV_inner.alpha[:, None],
        uac_paths.pv_boundaries.RIPV_inner.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.RIPV_inner.alpha.shape[0], 1)),
    )
)
uac_ripv_outer_coordinates = np.hstack(
    (
        uac_paths.pv_boundaries.RIPV_outer.alpha[:, None],
        uac_paths.pv_boundaries.RIPV_outer.beta[:, None],
        np.zeros((uac_paths.pv_boundaries.RIPV_outer.alpha.shape[0], 1)),
    )
)
uac_laa_coordinates = np.hstack(
    (
        uac_paths.laa_boundary.alpha[:, None],
        uac_paths.laa_boundary.beta[:, None],
        np.zeros((uac_paths.laa_boundary.alpha.shape[0], 1)),
    )
)
uac_rspv_inner_mesh = pv.PolyData(uac_rspv_inner_coordinates)
uac_rspv_outer_mesh = pv.PolyData(uac_rspv_outer_coordinates)
uac_lspv_inner_mesh = pv.PolyData(uac_lspv_inner_coordinates)
uac_lspv_outer_mesh = pv.PolyData(uac_lspv_outer_coordinates)
uac_lipv_inner_mesh = pv.PolyData(uac_lipv_inner_coordinates)
uac_lipv_outer_mesh = pv.PolyData(uac_lipv_outer_coordinates)
uac_ripv_inner_mesh = pv.PolyData(uac_ripv_inner_coordinates)
uac_ripv_outer_mesh = pv.PolyData(uac_ripv_outer_coordinates)
uac_laa_mesh = pv.PolyData(uac_laa_coordinates)

plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(roof_uac_mesh, style="wireframe", color="grey")
plotter.add_mesh(anterior_uac_mesh, style="wireframe", color="grey")
plotter.add_mesh(septal_uac_mesh, style="wireframe", color="grey")
plotter.add_mesh(lateral_uac_mesh, style="wireframe", color="grey")
plotter.add_mesh(posterior_uac_mesh, style="wireframe", color="grey")
plotter.add_points(uac_rspv_inner_mesh, color="red", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_rspv_outer_mesh, color="blue", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_lspv_inner_mesh, color="red", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_lspv_outer_mesh, color="blue", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_lipv_inner_mesh, color="red", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_lipv_outer_mesh, color="blue", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_ripv_inner_mesh, color="red", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_ripv_outer_mesh, color="blue", point_size=10, render_points_as_spheres=True)
plotter.add_points(uac_laa_mesh, color="green", point_size=10, render_points_as_spheres=True)
plotter.view_xy()
plotter.show()

Widget(value='<iframe src="http://localhost:32903/index.html?ui=P_0x7f0c44138550_4&reconnect=auto" class="pyvi…

In [ ]:
plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(roof_uac_mesh, style="wireframe", color="grey")
plotter.add_mesh(anterior_uac_mesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()

In [ ]:
combined_vertices = np.vstack((roof_uac_mesh.points, anterior_uac_mesh.points))
combined_simplices = np.vstack(
    (
        roof_uac_mesh.faces.reshape((-1, 4))[:, 1:],
        anterior_uac_mesh.faces.reshape((-1, 4))[:, 1:] + roof_uac_mesh.n_points,
    )
)
combined_uac_mesh = pv.PolyData.from_regular_faces(combined_vertices, combined_simplices).clean()
combined_uac_mesh = combined_uac_mesh.smooth(
    n_iter=500,
    relaxation_factor=0.1,
    boundary_smoothing=True,
    feature_smoothing=True,
    edge_angle=120,
    feature_angle=45,
)

plotter = pv.Plotter(window_size=[900, 900])
plotter.add_mesh(combined_uac_mesh, style="wireframe", color="grey")
plotter.view_xy()
plotter.show()